In [26]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from google.colab import drive

# 1 Load Data

In [27]:
drive.mount("/content/gdrive")
df = pd.read_csv("/content/gdrive/My Drive/T1_DV/DS_cleaned_final.csv")
df.info()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12113 entries, 0 to 12112
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12113 non-null  int64  
 1   city                    12113 non-null  object 
 2   city_development_index  12113 non-null  float64
 3   gender                  12113 non-null  object 
 4   relevent_experience     12113 non-null  object 
 5   education_level         12113 non-null  int64  
 6   major_discipline        12113 non-null  object 
 7   experience              12113 non-null  int64  
 8   company_size            12113 non-null  int64  
 9   company_type            12113 non-null  object 
 10  last_new_job            12113 non-null  int64  
 11  training_hours          12113 non-null  int64  
 12  target       

# 2 Data Manipulation
### The object is to investigate the relationship between Education Backgrounds and the Company Types

In [28]:
#### Get all categories of companies
cate = ['Early Stage Startup','Funded Startup','NGO','Other','Public Sector','Pvt Ltd','Early Stage Startup']
cate

['Early Stage Startup',
 'Funded Startup',
 'NGO',
 'Other',
 'Public Sector',
 'Pvt Ltd',
 'Early Stage Startup']

In [29]:
#### Using groupby aggregate function to count the employee number within each company
df.loc[df["education_level"]==0].groupby("company_type").agg("nunique")

,enrollee_id,city,city_development_index,gender,relevent_experience,education_level,major_discipline,experience,company_size,last_new_job,training_hours,target
company_type,,,,,,,,,,,,
Early Stage Startup,6,5,4,3,2,1,1,5,3,3,6,2
Funded Startup,7,7,6,2,2,1,1,4,3,3,7,2
NGO,2,2,1,1,2,1,1,2,1,2,2,1
Other,1,1,1,1,1,1,1,1,1,1,1,1
Public Sector,4,3,3,2,2,1,1,4,3,2,4,2
Pvt Ltd,36,19,18,2,2,1,1,12,8,6,28,2


In [30]:
#### Save company count into 5 lists by education backgrounds
con0 = [i for i in df.loc[df["education_level"]==0].groupby("company_type").agg("nunique")["enrollee_id"]]
con1 = [i for i in df.loc[df["education_level"]==1].groupby("company_type").agg("nunique")["enrollee_id"]]
con2 = [i for i in df.loc[df["education_level"]==2].groupby("company_type").agg("nunique")["enrollee_id"]]
con3 = [i for i in df.loc[df["education_level"]==3].groupby("company_type").agg("nunique")["enrollee_id"]]
con4 = [i for i in df.loc[df["education_level"]==4].groupby("company_type").agg("nunique")["enrollee_id"]]

In [31]:
#### Calculate the proportion of the people working in a certain company among all people.
# 5 rates would be stored into each list.
# For Education n, rate = Company Type/All where employees are with n degree
rate0 = [i/np.sum(con0) for i in con0]
rate1 = [i/np.sum(con1) for i in con1]
rate2 = [i/np.sum(con2) for i in con2]
rate3 = [i/np.sum(con3) for i in con3]
rate4 = [i/np.sum(con4) for i in con4]
# Concate the whole list to the first value of rate, in order to make the pie chart complete
# Store values into dataframes 'df'
df0 = pd.DataFrame(dict({"r":np.concatenate((rate0, [rate0[0]]))}))
df1 = pd.DataFrame(dict({"r":np.concatenate((rate1, [rate1[0]]))}))
df2 = pd.DataFrame(dict({"r":np.concatenate((rate2, [rate2[0]]))}))
df3 = pd.DataFrame(dict({"r":np.concatenate((rate3, [rate3[0]]))}))
df4 = pd.DataFrame(dict({"r":np.concatenate((rate4, [rate4[0]]))}))

In [15]:
fig = go.Figure()
fig.add_trace(go.Scatterpolar(r=df0["r"], theta=cate, fill="toself", name="Primary School"))
fig.add_trace(go.Scatterpolar(r=df1["r"], theta=cate, fill="toself", name="High School"))
fig.add_trace(go.Scatterpolar(r=df2["r"], theta=cate, fill="toself", name="Bachelor"))
fig.add_trace(go.Scatterpolar(r=df3["r"], theta=cate, fill="toself", name="Master"))
fig.add_trace(go.Scatterpolar(r=df4["r"], theta=cate, fill="toself", name="Phd"))

fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0,0.8])), showlegend=True)
fig.show()

In [32]:
#### It's recognised that the radar chart has extreme pointers which make the other dimensions less easy to be observed.
# This was because the rates were calculated based on employees from all types of company (There are more Pvt Ltd than others), instead of a horizontal comparison within each sector (company type)
# The solution is to calculate rates based on employees from the same company type: within each company type, the proportion of employees from different education backgrounds
l = [con0, con1, con2, con3, con4]

# Get the count of employees of all education backgrounds by company type
s0 = np.sum([l[i][0] for i in np.arange(5)])
s1 = np.sum([l[i][1] for i in np.arange(5)])
s2 = np.sum([l[i][2] for i in np.arange(5)])
s3 = np.sum([l[i][3] for i in np.arange(5)])
s4 = np.sum([l[i][4] for i in np.arange(5)])
s5 = np.sum([l[i][5] for i in np.arange(5)])

s = [s0, s1, s2, s3, s4, s5]

# Calculate the proportion of the people working in different company types among people from the same company type by education background.
# For Company Type n, rate = education/All where employees work in n
p0 = [con0[i]/s[i] for i in np.arange(6)]
p1 = [con1[i]/s[i] for i in np.arange(6)]
p2 = [con2[i]/s[i] for i in np.arange(6)]
p3 = [con3[i]/s[i] for i in np.arange(6)]
p4 = [con4[i]/s[i] for i in np.arange(6)]

# Store values into dataframes 'd'
d0 = pd.DataFrame(dict({"r":np.concatenate((p0, [p0[0]]))}))
d1 = pd.DataFrame(dict({"r":np.concatenate((p1, [p1[0]]))}))
d2 = pd.DataFrame(dict({"r":np.concatenate((p2, [p2[0]]))}))
d3 = pd.DataFrame(dict({"r":np.concatenate((p3, [p3[0]]))}))
d4 = pd.DataFrame(dict({"r":np.concatenate((p4, [p4[0]]))}))

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatterpolar(r=d2["r"], theta=cate, fill="toself", name="Bachelor", marker=dict(color="steelblue")))
fig.add_trace(go.Scatterpolar(r=d3["r"], theta=cate, fill="toself", name="Master", marker=dict(color="paleturquoise")))
fig.add_trace(go.Scatterpolar(r=d1["r"], theta=cate, fill="toself", name="High School", marker=dict(color="gold")))
fig.add_trace(go.Scatterpolar(r=d0["r"], theta=cate, fill="toself", name="Primary School", marker=dict(color="darkorange")))
fig.add_trace(go.Scatterpolar(r=d4["r"], theta=cate, fill="toself", name="Phd", marker=dict(color="tomato")))

fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), polar=dict(radialaxis=dict(visible=True, range=[0,0.75])), showlegend=True)
fig.show()